In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
# inicializa o programa sem aparecer na tela, em segundo plano
chrome_options = Options()
chrome_options.headless = True
navegador = webdriver.Chrome(options=chrome_options)

# passo 1: pegar a cotação das moedas
# dolar
navegador.get('https://www.google.com/search?q=cotacao+dolar&oq=cotacao+dolar&aqs=chrome..69i57j35i39j0i512l3j0i433i512j0i512l4.1498j0j9&sourceid=chrome&ie=UTF-8')
cot_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# euro
navegador.get('https://www.google.com/search?q=cotacao+euro&oq=cotacao+euro&aqs=chrome..69i57j0i512l4j0i10i512j0i512l4.1450j0j7&sourceid=chrome&ie=UTF-8')
cot_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20314%2C92.')
cot_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cot_ouro = cot_ouro.replace(',', '.')

navegador.quit()

print('Dólar: R$' + cot_dolar)
print('Euro: R$' + cot_euro)
print('Ouro: R$' + cot_ouro)

Dólar: R$5.1487
Euro: R$5.5691999999999995
Ouro: R$317.52


In [4]:
# passo 2: importar e atualizar a base de dados
df = pd.read_excel('Produtos.xlsx')

df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [5]:
# atualizar a cotação das moedas no dataframe
df.loc[df['Moeda'] == 'Dólar', 'Cotação'] = float(cot_dolar)    # loc[linha, coluna]
df.loc[df['Moeda'] == 'Euro', 'Cotação'] = float(cot_euro)
df.loc[df['Moeda'] == 'Ouro', 'Cotação'] = float(cot_ouro)

df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.1487,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.5692,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.1487,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.1487,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.5692,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.1487,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,317.5200,7000.00,1.15,8050.000


In [6]:
# atualizar o preço de compra e preço de venda
df['Preço de Compra'] = df['Preço Original'] * df['Cotação']
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']

# formatar os preços
df['Preço de Compra'], df['Preço de Venda'] = df['Preço de Compra'].map('R${:.2f}'.format), df['Preço de Venda'].map('R${:.2f}'.format)
df['Cotação'] = df['Cotação'].map('R${:.2f}'.format)

df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,R$5.15,R$5148.65,1.40,R$7208.11
1,Carro Renault,4500.00,Euro,R$5.57,R$25061.40,2.00,R$50122.80
2,Notebook Dell,899.99,Dólar,R$5.15,R$4633.78,1.70,R$7877.42
3,IPhone,799.00,Dólar,R$5.15,R$4113.81,1.70,R$6993.48
4,Carro Fiat,3000.00,Euro,R$5.57,R$16707.60,1.90,R$31744.44
5,Celular Xiaomi,480.48,Dólar,R$5.15,R$2473.85,2.00,R$4947.69
6,Joia 20g,20.00,Ouro,R$317.52,R$6350.40,1.15,R$7302.96


In [8]:
# passo 3: exportar a base de dados
df.to_excel('Produtos Novo.xlsx', index=False)   # nome do arquivo a ser criado | index=false para não exportar o index junto

In [9]:
df_novo = pd.read_excel('Produtos Novo.xlsx')
df_novo

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,R$5.15,R$5148.65,1.40,R$7208.11
1,Carro Renault,4500.00,Euro,R$5.57,R$25061.40,2.00,R$50122.80
2,Notebook Dell,899.99,Dólar,R$5.15,R$4633.78,1.70,R$7877.42
3,IPhone,799.00,Dólar,R$5.15,R$4113.81,1.70,R$6993.48
4,Carro Fiat,3000.00,Euro,R$5.57,R$16707.60,1.90,R$31744.44
5,Celular Xiaomi,480.48,Dólar,R$5.15,R$2473.85,2.00,R$4947.69
6,Joia 20g,20.00,Ouro,R$317.52,R$6350.40,1.15,R$7302.96
